In [2]:
import re
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

In [2]:
from random import randrange
pr_file_path="C:\\apache-tomcat-7.0.34\\webapps\\assignment5"
os.chdir('C:\\Program Files\\MongoDB\\Server\\3.2\\bin')

out = open(pr_file_path+'/sample_data.csv', 'w',newline='')
output = csv.writer(out)

productList = ['Sony - 55 inch Class','Microsoft TV','Sony - 77 inch Class','Samsung - 55 inch Class','Samsung Smart TV 50"','JBL Bar 5.1','Bose Surround Speakers 700','JBL Cinema 610','Denon Home Theater Soundbar','Bose Solo 5','Samsung Galaxy S20','OnePlus 8','Google Pixel 3a','Google Pixel 4 XL','Samsung Galaxy S10','HP Chromebook 14','HP Stream','Lenovo IdeaPad 3','ASUS Chromebook Laptop','Lenovo T460s','Toshiba 49" Class LED TV','HP 14" Touch Screen Laptop','Lenovo Yoga 730 2 in 1 15.6" Touch Screen Laptop','Samsung Harmon Kardon 7.1.4 Channel Soundbar']
output.writerow(['userName','productName','reviewRating'])
for i in range(10):
    output.writerow(["customer" + str(randrange(1,10)), productList[randrange(len(productList))], randrange(1,5)])
out.close()

os.system(r'mongoimport --db Assignment2CustomerReviews --collection myReviews --type=csv --headerline --drop '+pr_file_path+'\sample_data.csv')




0

In [4]:
pr_file_path="C:\\apache-tomcat-7.0.34\\webapps\\assignment5"
os.chdir('C:\\Program Files\\MongoDB\\Server\\3.2\\bin')
os.system(r'mongoexport --db Assignment2CustomerReviews --collection myReviews --type=csv --fields userName,productName,reviewRating >'+pr_file_path+'\mongodata_train.csv')

with open(pr_file_path+"/mongodata_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open(pr_file_path+"/mongodata_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser(pr_file_path+'/mongodata_test.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=4)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open(pr_file_path+'/output.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


customer6 ['"Toshiba 49"" Class LED TV"', 'Denon Home Theater Soundbar', 'Lenovo IdeaPad 3', '"HP 14"" Touch Screen Laptop"']
customer4 ['"HP 14"" Touch Screen Laptop"', 'Denon Home Theater Soundbar', 'Google Pixel 4 XL', 'ASUS Chromebook Laptop']
customer8 ['Denon Home Theater Soundbar', 'Lenovo IdeaPad 3', 'Google Pixel 4 XL', 'ASUS Chromebook Laptop']
customer3 ['"Toshiba 49"" Class LED TV"', 'Google Pixel 4 XL', 'Lenovo IdeaPad 3', '"HP 14"" Touch Screen Laptop"']
customer1 ['"Toshiba 49"" Class LED TV"', 'Denon Home Theater Soundbar', 'Google Pixel 4 XL', 'Lenovo IdeaPad 3']
customer7 ['"HP 14"" Touch Screen Laptop"', 'Denon Home Theater Soundbar', '"Toshiba 49"" Class LED TV"', 'Lenovo IdeaPad 3']
